In [15]:
import numpy as np
#from matplotlib import pyplot as plt
import pandas as pd
import math
import serial
import time
import asyncio
import binascii
#import websockets
from ast import literal_eval

In [16]:
#initializations
controlParams = {
  "ci": 8,
  "dpl": 251,
  "att_mtu": 247,
  "tp":0,
  "phy":2,
  "ext":False,
  "max_ce":6,
  "CE_dpl_h":2.5,
  "CE_dpl_l":4, 
  "nce":2,
  "power":0,
  "read":500,
  "payload":244,
  "nperi":5,
  "expectedLat":100
}
timer = 20
controlParamsList = ["ci","dpl","att_mtu","tp","phy","read"]
windowSize=20
window=0
count=0

In [17]:
nrf = serial.Serial('COM3', 115200, writeTimeout = 0) # enable this to read from serial port

SerialException: could not open port 'COM3': PermissionError(13, 'Access is denied.', None, 5)

In [18]:
test = 'ci:46\n'
nrf.write(test.encode())

6

In [8]:
test = 'restart:21\n'
nrf.write(test.encode())

11

In [12]:
read()

NameError: name 'read' is not defined

In [104]:

stopRead()


In [5]:
nrf.close()

NameError: name 'nrf' is not defined

In [19]:

#spork's model for estimating link quality in terms of number of connection events from latency - improving timeliness of BLE
def calcNumbOfConnEvents(latency,ce,payload,ATT_MTU,ci):
    return math.ceil((latency-2*ce)/(2*math.ceil(payload/ATT_MTU)*ci))
#spork's model for estimating link quality in terms of number of connection events from latency (HCI event based) from master
def nceEst_hci_master(latency,max_ce,payload,ATT_MTU,ci):
    return math.ceil((latency-max_ce)/(math.ceil(payload/ATT_MTU)*ci))
#spork's model for estimating link quality in terms of number of connection events from latency (HCI event based) from slave
def nceEst_hci_slave(latency,max_ce,payload,ATT_MTU,ci):
    return (math.ceil((latency-max_ce)/(math.ceil(payload/ATT_MTU)*ci))-1)
#spork's model for estimating link quality in terms of number of connection events from latency (RTT based)
def nceEst_RTT(latency,max_ce,payload,ATT_MTU,ci):
    return math.ceil((latency-2*max_ce)/(2*math.ceil(payload/ATT_MTU)*ci))
#Kalai's model for estimating link quality in terms of number of connection events from latency (RTT based)
def nceEst_kalai(latency,max_ce,payload,ATT_MTU,ci,nperi):
    return (math.ceil((latency-max_ce)/(math.ceil(payload/ATT_MTU)*ci*nperi*0.5)))
# estimation of ci using spork's algo
def estimateCI_spork(expectedLat,max_ce,payload,ATT_MTU,max_nce):
    return math.floor((expectedLat - max_ce)/(math.ceil(payload/ATT_MTU)*max_nce))
# estimation of ci using kalai's algo
def estimateCI_kalai(expectedLat,max_ce,payload,ATT_MTU,max_nce,nperi):
    return (math.floor((expectedLat - max_ce)/(math.ceil(payload/ATT_MTU)*max_nce*2)))
# estimation of ci using kalai's ML model
'''def CIOptimizer_data(dpl,nperi,latencyTh,interference):
    data = pd.read_csv('data_4devices_wo.csv')
    #query = "dpl =="+dpl+' & nperi=='+nperi+' & latencyTh=='+latencyTh+' & nce=='+nce
    if(interference==2):
        query = 'dpl =='+str(dpl)+' & nperi=='+str(nperi)+' & latency<'+str(latencyTh)
    if(dpl==1 and interference==2):
        query = 'nperi=='+str(nperi)+' & latency<'+str(latencyTh)
    else:
        query = 'dpl =='+str(dpl)+' & nperi=='+str(nperi)+' & latency<'+str(latencyTh)+' & noise=='+str(interference)  
    print(query)
    data_dpl = data.query(query)
    #data_dpl.sort_values(by=['latency'])
    #print(data_dpl)
    return data_dpl'''

def CIOptimizer_data(dpl,nperi,latencyTh,interference,data):
    #query = 'nperi=='+str(nperi)+' & latency<'+str(latencyTh)
    if(interference==2):
        query = 'dpl =='+str(dpl)+' & nperi=='+str(nperi)+' & latency<'+str(latencyTh)
    else:
        query = 'dpl =='+str(dpl)+' & nperi=='+str(nperi)+' & latency<'+str(latencyTh)+' & noise=='+str(interference)
    print(query)
    data_dpl = data.query("latency<"+str(latencyTh))
    #data_dpl.sort_values(by=['latency'])
    #print(data_dpl)
    if(dpl):
        data_dpl=data_dpl.query("dpl=="+str(dpl))
    if(nperi):
        data_dpl=data_dpl.query("nperi=="+str(nperi))
    data_dpl=data_dpl.query("noise=="+str(interference))
    return data_dpl

In [5]:
print(nceEst_hci_master(130,6,244,247,10))

13


In [6]:
ci=[]
opt=True
optSize = 0
ind=0
optdata = pd.read_csv('model_input_ver2.csv')
print(optdata)
while ind<20:
    if(not opt and optSize<len(ci)):
        print('update ci ',ci[optSize])
        optSize=optSize+1
        if(optSize==len(ci)):
            print('reset')
            optSize=0
            ci=[]
    if(opt):
        opt=False
        df = CIOptimizer_data(251,5,100,0,optdata)
        df = df.sort_values(['nce'])
        ci=df['ci'].unique()
        print(ci)
        print(df)
    ind=ind+1
    print(ind)
    time.sleep(1)

      dpl  nperi         ci         pow  latency  nce  noise
0      27      3  10.000000  556.758183      119   12      1
1      27      3  15.000000  494.819065       95    6      1
2      27      3  20.000000  405.817338      205   10      1
3      27      3  25.000000  372.365782      190    8      1
4      27      3  30.000000  312.348810      378   13      1
...   ...    ...        ...         ...      ...  ...    ...
5856  251      5  10.071429  492.152949       77    8      0
5857  251      5  10.071429  492.152949       77    8      0
5858  251      5  10.071429  492.152949       77    8      0
5859  251      5  10.071429  509.306463       77    8      0
5860  251      5  10.071429  509.306463       77    8      0

[5861 rows x 7 columns]
dpl ==251 & nperi==5 & latency<100 & noise==0
[46.         15.         20.         13.         10.         11.
  8.          9.         10.07142857 12.        ]
      dpl  nperi    ci         pow  latency  nce  noise
2014  251      5  46.0  16

KeyboardInterrupt: 

In [7]:
nrf.close()

In [25]:
nrf = serial.Serial('COM3', 115200, writeTimeout = 0) # enable this to read from serial port

In [23]:
from ppk2_api.ppk2_api import PPK2_API
ppk2_test = PPK2_API("COM4")  # serial port will be different for you
ppk2_test.get_modifiers()
#ppk2_test.use_source_meter()  # set source meter mode
ppk2_test.use_ampere_meter()
#ppk2_test.set_source_voltage(3300)  # set source voltage in mV
ppk2_test.toggle_DUT_power("ON")  # enable DUT power

In [20]:
def readPower():
    ppk2_test.start_measuring()  # start measuring
    final = np.array([])
    power = np.array([])
    numberOfPeaks = np.array([])
    numberOfPeaks1 = np.array([])
    count=0
    counter = 0
    while(1):
        count=count+1
        counter=counter+1
        read_data = ppk2_test.get_data()
        if read_data != b'':
            samples = ppk2_test.get_samples(read_data)
            #print(f"Average of {len(samples)} samples is: {sum(samples)/len(samples)}uA")
            arr = np.array(samples)
            filtered = arr[arr>0]
            numberOfPeaks = arr[arr>4000]
            numberOfPeaks1 = arr[arr>1000]
            #print(filtered)
            if(len(filtered)):
                final = np.append(final, sum(filtered)/len(filtered))
            if(count==50):
                #print('size of numpy array ',final.size)  
                #print('average current is :',np.average(final))
                power = np.append(power, np.average(final))
                print(power)
                count=0
                final = np.array([])
            if(counter==50):
                #print('size of powerarray ',power.size) 
                ppk2_test.stop_measuring()
                #print('number of power measurements more than 4000',numberOfPeaks.size)
                #print('number of power measurements more than 1000',numberOfPeaks1.size)
                return np.max(power)
        time.sleep(0.001)  # lower time between sampling -> less samples read in one sampling period

In [21]:
print(readPower())

[1923.80378402]
1923.8037840234235


In [22]:
def read():
    nrf.flushInput()
    time.sleep(5)
    test = 'read:1\n'
    nrf.write(test.encode())
    time.sleep(3)
    test = 'rlat:1\n'
    nrf.write(test.encode())
    time.sleep(3)

def stopRead(): 
    test = 'slat:1\n'
    nrf.write(test.encode())
    time.sleep(3)
    test = 'stop:1\n'
    nrf.write(test.encode())
    time.sleep(3)
read()

In [28]:
def cmdToNrf(cmd):
    #nrf = serial.Serial('COM14', 115200) # enable this to read from serial port
    #print(cmd)
    test = cmd
    nrf.write(test.encode())
    time.sleep(20)
    read()
    time.sleep(5)
    #print('is port open :',nrf.is_open)
    #nrf.close()

In [29]:
def updateCI(ci,data):
    cmd = 'ci:'+str(ci)+'\n'
    #cmd = 'ci:'+str(ciEst_kalai_val)+'\n'
    #cmdToNrf('restart:1\n')
    #time.sleep(15)
    stopRead()
    #nrf.close()
    print('stopped read')
    test = 'slat:1\n'
    nrf.write(test.encode())
    time.sleep(3)
    test = 'restart:1\n'
    cmdToNrf(test)
    print('restarted')
    cmdToNrf(cmd)           
    controlParams['ci']=ci
    data.at[index,'est_ci'] = ci
    print('changed connection interval')
    #read()
    print('started read')
    return data

In [30]:
import pickle
pickle_in = open('interference2.pkl','rb')
interferenceIdentifier = pickle.load(pickle_in)

'''def verifyInterference(dpl,nperi,ci,latency,Rfactor):
    predInt = interferenceIdentifier.predict([[dpl,nperi,ci,latency,Rfactor]])
    print(predInt)
    if(predInt=='bad'):
        return 1
    else:
        return 0'''

def verifyInterference(dpl,nperi,ci,Rfactor):
    predInt = interferenceIdentifier.predict([[dpl,nperi,ci,Rfactor]])
    print(predInt)
    if(predInt=='bad'):
        return 1
    else:
        return 0

In [31]:
#initializations
controlParams = {
  "ci": 8,
  "dpl": 251,
  "att_mtu": 247,
  "tp":0,
  "phy":2,
  "ext":False,
  "max_ce":6,
  "CE_dpl_h":2.5,
  "CE_dpl_l":5, 
  "nce":2,
  "power":0,
  "read":500,
  "payload":244,
  "nperi":5,
  "expectedLat":100,
  "lat_high":120,
  "lat_low":80
}
timer = 20
controlParamsList = ["ci","dpl","att_mtu","tp","phy","read"]
windowSize=20
window=0
count=0

In [32]:
nrf.flushInput()

In [14]:
#old algo
index=0
data = pd.read_csv('template_ci_algo_1stJune_5Devices_Spork_Estimation_20.csv')
print(data.head(5))
data=data.astype(object)
nceEst_hci_master_list = []
nceEst_hci_slave_list = []
nceEst_RTT_list = []
nceEst_kalai_list = []
latency_list = []
interference_list = []
pow_list = []
power=readPower()
nrf.flushInput()
time.sleep(5)
while index<=1000:
    lat = np.array([])
    lat = nrf.readline()
    test = (lat.decode()).strip("x00\n")
    x = test.split(",")
    print(float(x[4].strip("\x00")))
    #print(bytes([12]))
    #lat1 = np.array([test])
    #lat1 = lat1.astype(np.int_)
    lat1 = int(x[0].strip("\x00"))
    #print(lat1)
    latency=lat1
    #print(latency)
    latency_list.append(latency)
    data.at[index,'nce'] = nceEst_hci_master_val
    nceEst_hci_master_val = nceEst_hci_master(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
    data.at[index,'nce'] = nceEst_hci_master_val
    nceEst_hci_slave_val = nceEst_hci_slave(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
    data.at[index,'nce_spork'] = nceEst_hci_slave_val
    nceEst_RTT_val = nceEst_RTT(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
    data.at[index,'nce_RTT'] = nceEst_RTT_val
    nceEst_kalai_val = nceEst_kalai(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'],controlParams['nperi'])
    data.at[index,'nce_kalai'] = nceEst_kalai_val
    nceEst_hci_master_list.append(nceEst_hci_master_val)
    nceEst_hci_slave_list.append(nceEst_hci_slave_val)
    nceEst_RTT_list.append(nceEst_RTT_val)
    nceEst_kalai_list.append(nceEst_kalai_val)
    #interference = verifyInterference(controlParams['dpl'],controlParams['nperi'],controlParams['ci'],latency,nceEst_hci_master_val)
    interference = verifyInterference(controlParams['dpl'],controlParams['nperi'],controlParams['ci'],nceEst_hci_master_val)
    interference_list.append(interference)
    window = window+1
    #print(nceEst_kalai_list)
    max_lat = max(latency_list)
    min_lat = min(latency_list)
    min_max_diff = max_lat-min_lat
    data.at[index,'latency'] = latency
    data.at[index,'lat_min_max_diff'] = min_max_diff
    latTh_latMax_diff = max_lat - controlParams['expectedLat']
    data.at[index,'latTh_latMax_diff'] = latTh_latMax_diff
    latTh_diff = latency - controlParams['expectedLat']
    #print('Latency fo')
    data.at[index,'latTh_lat_diff'] = latTh_diff
    data.at[index,'est_ci'] = controlParams['ci']
    data.at[index,'pow'] = power
    #--------------- spork's algorithm----------------------
    if(window == windowSize):
        window=0
        power=readPower()
        data.at[index,'pow'] = power
        nceEst_hci_master_max = max(nceEst_hci_master_list)
        data.at[index,'nce_all'] = nceEst_hci_master_max
        nceEst_hci_slave_max = max(nceEst_hci_slave_list)
        data.at[index,'nce_spork_all'] = nceEst_hci_slave_max
        nceEst_RTT_max = max(nceEst_RTT_list)
        data.at[index,'nce_RTT_all'] = nceEst_RTT_max
        nceEst_kalai_max = max(nceEst_kalai_list)
        data.at[index,'nce_kalai_all'] = nceEst_kalai_max
        #print(nceEst_hci_slave_max)
        print(nceEst_hci_master_list)
        print(nceEst_hci_slave_list)
        print(nceEst_RTT_list)
        print(nceEst_kalai_list)
        print(interference_list)
        ciEst_hci_master_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_master_max)
        data.at[index,'nce_ci'] = ciEst_hci_master_val
        ciEst_hci_slave_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_slave_max)
        data.at[index,'ci_spork'] = ciEst_hci_slave_val
        ciEst_RTT_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_RTT_max)
        data.at[index,'ci_RTT'] = ciEst_RTT_val
        ciEst_kalai_val = estimateCI_kalai(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_master_max,controlParams['nperi'])
        data.at[index,'ci_kalai'] = ciEst_kalai_val
        #print(latency)
        print(ciEst_hci_master_val)
        print(ciEst_hci_slave_val)
        print(ciEst_RTT_val)
        print(ciEst_kalai_val)
        ciFinal = ciEst_hci_master_val
        print('--------------------')
        if(ciFinal<=8):
            ciFinal = 8
        if(controlParams['ci']!=ciFinal):
            print('estimated connection intervals are ciEst_hci_master_val:',ciEst_hci_master_val,',ciEst_hci_slave_val:',ciEst_hci_slave_val,',ciEst_RTT_val:',ciEst_RTT_val,',ciEst_kalai_val:',ciEst_kalai_val)
            cmd = 'ci:'+str(ciFinal)+'\n'
            #cmd = 'ci:'+str(ciEst_kalai_val)+'\n'
            #cmdToNrf('restart:1\n')
            #time.sleep(15)
            stopRead()
            #nrf.close()
            print('stopped read')
            test = 'slat:1\n'
            nrf.write(test.encode())
            time.sleep(3)
            test = 'restart:1\n'
            cmdToNrf(test)
            print('restarted')
            cmdToNrf(cmd)           
            controlParams['ci']=ciFinal
            data.at[index,'est_ci'] = ciFinal
            data.at[index,'pow'] = power
            print('changed connection interval')
            #read()
            print('started read')
        nceEst_hci_master_list = []
        nceEst_hci_slave_list = []
        nceEst_RTT_list = []
        nceEst_kalai_list = []
        latency_list = []
        interference_list = []
    if(index%1==0):
        data.to_csv('template_ci_algo_1stJune_5Devices_Spork_Estimation_20.csv', encoding='utf-8', index=False)
        index=index+1
    #print('reading from uart')
    time.sleep(1)

   ind  finished    dpl  att_mtu   tp  phy  ci_initial  est_ci        pow  \
0  1.0       NaN  251.0    247.0  0.0  2.0         7.5       8  625.82254   
1  2.0       NaN  251.0    247.0  0.0  2.0         7.5       8  625.82254   
2  3.0       NaN  251.0    247.0  0.0  2.0         7.5       8  625.82254   
3  4.0       NaN  251.0    247.0  0.0  2.0         7.5       8  625.82254   
4  5.0       NaN  251.0    247.0  0.0  2.0         7.5       8  625.82254   

   pow_avg  ...  nce_ci  nce_spork  nce_spork_all  ci_spork  nce_RTT  \
0      NaN  ...     NaN          8            NaN       NaN        4   
1      NaN  ...     NaN          8            NaN       NaN        4   
2      NaN  ...     NaN          8            NaN       NaN        4   
3      NaN  ...     NaN          8            NaN       NaN        4   
4      NaN  ...     NaN          8            NaN       NaN        4   

   nce_RTT_all  ci_RTT  nce_kalai  nce_kalai_all  ci_kalai  
0          NaN     NaN          4          

KeyboardInterrupt: 

In [14]:
import pickle
pickle_in = open('test.pkl','rb')
regressor = pickle.load(pickle_in)
pickle_in = open('DT_Class.pkl','rb')
classifier = pickle.load(pickle_in)

In [33]:
#initializations
controlParams = {
  "ci": 8,
  "dpl": 251,
  "att_mtu": 247,
  "tp":0,
  "phy":2,
  "ext":False,
  "max_ce":6,
  "CE_dpl_h":2.5,
  "CE_dpl_l":5, 
  "nce":2,
  "power":0,
  "read":500,
  "payload":244,
  "nperi":5,
  "expectedLat":100,
  "lat_high":120,
  "lat_low":80,
  "expected_cat":1
}
timer = 20
controlParamsList = ["ci","dpl","att_mtu","tp","phy","read"]
windowSize=15
window=0
count=0

In [34]:
def validateEst_CI(dpl,nperi,est_ci,expected_cat):
    predCat = classifier.predict([[dpl,nperi,est_ci]])
    if(expected_cat==predCat[0]):
        return True
    else:
        return False

In [35]:
def estimateML_CI(dpl,nperi,expectedLat,expRet):
    return regressor.predict([[dpl,nperi,expectedLat,expRet]])



In [36]:
def estimateML_CI(dpl,nperi,expectedLat,expRet,expected_cat):
    est_ci = regressor.predict([[dpl,nperi,expectedLat,expRet]])
    isValid = validateEst_CI(dpl,nperi,est_ci,expected_cat)
    return est_ci,isValid

In [37]:
read()

In [38]:
data = pd.read_csv('template_ci_algo_9thMay_5Devices_ML_Estimation_esp32_wi.csv')
for index, row in data.iterrows():
    if(data.at[index,'finished']==True):
        break
index

0

In [39]:
#ML algo
index=0
data = pd.read_csv('template_ci_algo_1stJuly_2Devices_ML_Estimation_15.csv')
'''for index, row in data.iterrows():
    if(data.at[index,'finished']==False):
        break'''
print(data.head(5))
data=data.astype(object)
optdata = pd.read_csv('model_input_4.csv')
nceEst_hci_master_list = []
nceEst_hci_slave_list = []
nceEst_RTT_list = []
nceEst_kalai_list = []
latency_list = []
pow_list = []
ci=[]
power=readPower()
blacklistCI=[]
expRet = 2
opt=False
optSize = 0
ind=0
estCI=8
#ppk2_test.start_measuring()
nrf.flushInput()
spork=False
first=True
time.sleep(5)
isCIValid=True
while index<=1000:
    time.sleep(1)
    lat = np.array([])
    lat = nrf.readline()
    if (lat != b'' or lat!=''):
        test = (lat.decode()).strip("x00\n")
        x = test.split(",")
        print(x)
        print(float(x[4].strip("\x00")))
        #print(bytes([12]))
        #lat1 = np.array([test])
        #lat1 = lat1.astype(np.int_)
        lat1 = int(x[0].strip("\x00"))
        #print(lat1)
        latency=lat1
        #print(latency)
        latency_list.append(latency)
        nceEst_hci_master_val = nceEst_hci_master(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
        data.at[index,'nce'] = nceEst_hci_master_val
        nceEst_hci_slave_val = nceEst_hci_slave(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
        data.at[index,'nce_spork'] = nceEst_hci_slave_val
        nceEst_RTT_val = nceEst_RTT(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
        data.at[index,'nce_RTT'] = nceEst_RTT_val
        nceEst_kalai_val = nceEst_kalai(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'],controlParams['nperi'])
        data.at[index,'nce_kalai'] = nceEst_kalai_val
        nceEst_hci_master_list.append(nceEst_hci_master_val)
        nceEst_hci_slave_list.append(nceEst_hci_slave_val)
        nceEst_RTT_list.append(nceEst_RTT_val)
        nceEst_kalai_list.append(nceEst_kalai_val)
        if(isCIValid):
            window = window+1
        #print(nceEst_kalai_list)
        max_lat = max(latency_list)
        min_lat = min(latency_list)
        min_max_diff = max_lat-min_lat
        data.at[index,'latency'] = latency
        data.at[index,'lat_min_max_diff'] = min_max_diff
        latTh_latMax_diff = max_lat - controlParams['expectedLat']
        data.at[index,'latTh_latMax_diff'] = latTh_latMax_diff
        latTh_diff = latency - controlParams['expectedLat']
        print('Latency:',latency)
        data.at[index,'latTh_lat_diff'] = latTh_diff
        data.at[index,'est_ci'] = controlParams['ci']
        data.at[index,'pow'] = power
        ciFinal=controlParams['ci']
        #--------------- spork's algorithm----------------------
        if(window == windowSize):
            badLat = [e for e in latency_list if e>controlParams['expectedLat']]
            latCount = len(badLat)
            print('bad latCount :',badLat)
            lqf=len(badLat)/len(latency_list)
            power=readPower()
            data.at[index,'pow'] = power
            '''pow_avg = np.average(pow_list)
            data.at[index,'pow_avg'] = pow_avg'''
            nceEst_hci_master_max = max(nceEst_hci_master_list)
            data.at[index,'nce_all'] = nceEst_hci_master_max
            nceEst_hci_slave_max = max(nceEst_hci_slave_list)
            data.at[index,'nce_spork_all'] = nceEst_hci_slave_max
            nceEst_RTT_max = max(nceEst_RTT_list)
            data.at[index,'nce_RTT_all'] = nceEst_RTT_max
            nceEst_kalai_max = max(nceEst_kalai_list)
            data.at[index,'nce_kalai_all'] = nceEst_kalai_max
            #print(nceEst_hci_slave_max)
            print(nceEst_hci_master_list)
            print(nceEst_hci_slave_list)
            print(nceEst_RTT_list)
            print(nceEst_kalai_list)
            ciEst_hci_master_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_master_max)
            data.at[index,'nce_ci'] = ciEst_hci_master_val
            ciEst_hci_slave_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_slave_max)
            data.at[index,'ci_spork'] = ciEst_hci_slave_val
            ciEst_RTT_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_RTT_max)
            data.at[index,'ci_RTT'] = ciEst_RTT_val
            ciEst_kalai_val = estimateCI_kalai(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_master_max,controlParams['nperi'])
            data.at[index,'ci_kalai'] = ciEst_kalai_val
            if(lqf > 0.1):
                currentCI = controlParams['ci']
                if (not (currentCI in blacklistCI) and lqf>0.1):
                    blacklistCI.append(currentCI)
                    print ("CI added to blacklist:",currentCI)
                opt=True
                # increase link quality by allowing more retransmissions at the cost of increased power
                expRet = expRet+1
                estCI,isCIValid=estimateML_CI(controlParams['dpl'],controlParams['nperi'],controlParams['expectedLat']-20,expRet,controlParams['expected_cat'])
                if(estCI in blacklistCI):
                    expRet = expRet+1 
                    estCI,isCIValid=estimateML_CI(controlParams['dpl'],controlParams['nperi'],controlParams['expectedLat']-20,expRet,controlParams['expected_cat'])
                estCI=estCI[0]
                #data = updateCI(estCI[0],data)
                print('estimatedCI:',estCI)
            if(lqf < 0.05):
                opt=True
                # Decrease link quality by allowing less retransmissions at the cost of decreased power
                if(not expRet<2):
                    expRet = expRet-1
                estCI,isCIValid=estimateML_CI(controlParams['dpl'],controlParams['nperi'],controlParams['expectedLat']-20,expRet,controlParams['expected_cat'])
                estCI=estCI[0]
                #data = updateCI(estCI[0],data)
                print('ML_estimatedCI:',estCI)
            #print(latency)
            print('spork_estimatedCI:',ciEst_hci_master_val)
            print(ciEst_hci_slave_val)
            print(ciEst_RTT_val)
            print(ciEst_kalai_val)
            if(not isCIValid):
                print('estimatedCI not valid:',estCI)
                continue
            if(ciFinal<controlParams['expectedLat']/2 and not (estCI in blacklistCI)):
                ciFinal=estCI
            if(ciFinal<=8):
                ciFinal = 8
            isCIValid=True
            #if(opt and spork):
            if(controlParams['ci']!=ciFinal):
                print('estimated connection intervals are ciEst_hci_master_val:',ciEst_hci_master_val,',ciEst_hci_slave_val:',ciEst_hci_slave_val,',ciEst_RTT_val:',ciEst_RTT_val,',ciEst_kalai_val:',ciEst_kalai_val)
                data = updateCI(ciFinal,data)
            nceEst_hci_master_list = []
            nceEst_hci_slave_list = []
            nceEst_RTT_list = []
            nceEst_kalai_list = []
            latency_list = []
            window=0
        data.at[index,'finished'] = True
        if(index%1==0):
            data.to_csv('template_ci_algo_1stJuly_2Devices_ML_Estimation_15.csv', encoding='utf-8', index=False)
            index=index+1
    #print('reading from uart')
    time.sleep(1)

   ind  finished    dpl  att_mtu   tp  phy  ci_initial  est_ci         pow  \
0  1.0      True  251.0    247.0  8.0  2.0         7.5     8.0  520.917679   
1  2.0      True  251.0    247.0  8.0  2.0         7.5     8.0  520.917679   
2  3.0      True  251.0    247.0  8.0  2.0         7.5     8.0  520.917679   
3  4.0      True  251.0    247.0  8.0  2.0         7.5     8.0  520.917679   
4  5.0      True  251.0    247.0  8.0  2.0         7.5     8.0  520.917679   

   pow_avg  ...  nce_ci  nce_spork  nce_spork_all  ci_spork  nce_RTT  \
0      NaN  ...     NaN          9            NaN       NaN        5   
1      NaN  ...     NaN          9            NaN       NaN        5   
2      NaN  ...     NaN          9            NaN       NaN        5   
3      NaN  ...     NaN          9            NaN       NaN        5   
4      NaN  ...     NaN          9            NaN       NaN        5   

   nce_RTT_all  ci_RTT  nce_kalai  nce_kalai_all  ci_kalai  
0          NaN     NaN          4    

PermissionError: [Errno 13] Permission denied: 'template_ci_algo_1stJuly_2Devices_ML_Estimation_15.csv'

In [32]:
pred = regressor.predict([[251,5,100,12]])
print(pred)

NameError: name 'regressor' is not defined

In [21]:
stopRead()

In [ ]:
read()

In [31]:
def estimateML_CI(dpl,nperi,expectedLat,expRet):
    return regressor.predict([[251,5,100,12]])

In [21]:
expRet=2
#latency=[233,100,90,90,90,123,234]
latency=[90,100,90,90,90,89,100]
badLat = [e for e in latency if e>100]
lqf=len(badLat)/len(latency)
def optCI(lqf,expRet):
    if (lqf > 0.2):
        opt=True
        # increase link quality by allowing more retransmissions at the cost of increased power
        expRet = expRet+1
        estCI=estimateML_CI(controlParams['dpl'],controlParams['nperi'],controlParams['expectedLat'],expRet)
        print('estimatedCI:',estCI)
    if(lqf == 0):
        opt=True
        # Decrease link quality by allowing less retransmissions at the cost of decreased power
        expRet = expRet+1
        estCI=estimateML_CI(controlParams['dpl'],controlParams['nperi'],controlParams['expectedLat'],expRet)
        print('estimatedCI:',estCI[0])
optCI(lqf,expRet)

estimatedCI: 9.0


In [28]:
#initializations
controlParams = {
  "ci": 8,
  "dpl": 251,
  "att_mtu": 247,
  "tp":0,
  "phy":2,
  "ext":False,
  "max_ce":6,
  "CE_dpl_h":2.5,
  "CE_dpl_l":5, 
  "nce":2,
  "power":0,
  "read":500,
  "payload":244,
  "nperi":5,
  "expectedLat":100,
  "lat_high":120,
  "lat_low":80
}
timer = 20
controlParamsList = ["ci","dpl","att_mtu","tp","phy","read"]
windowSize=120
window=0
count=0

In [ ]:
read()

In [ ]:
#data algo
index=0
data = pd.read_csv('template_ci_algo_21stMay_5Devices_Data_Estimation_120_1.csv')
'''for index, row in data.iterrows():
    if(data.at[index,'finished']==False):
        break'''
print(data.head(5)) 
data=data.astype(object)
optdata = pd.read_csv('model_input_ver2.csv')
nceEst_hci_master_list = []
nceEst_hci_slave_list = []
nceEst_RTT_list = []
nceEst_kalai_list = []
latency_list = []
pow_list = []
ci=[]
power=readPower()
blacklistCI=[]
expRet = 2
opt=False
optSize = 0
ind=0
estCI=8
#ppk2_test.start_measuring()
nrf.flushInput()
spork=False
first=True
time.sleep(5)
df = CIOptimizer_data(controlParams['dpl'],controlParams['nperi'],controlParams['expectedLat'],0,optdata)
print(df)
df = df.sort_values(['nce'])
ci=df['ci'].unique()
print(ci)
print('*****************')
while index<=2000:
    time.sleep(1)
    lat = np.array([])
    lat = nrf.readline()
    if (lat != b'' or lat!=''):
        test = (lat.decode()).strip("x00\n")
        x = test.split(",")
        print(float(x[4].strip("\x00")))
        #print(bytes([12]))
        #lat1 = np.array([test])
        #lat1 = lat1.astype(np.int_)
        lat1 = int(x[0].strip("\x00"))
        #print(lat1)
        latency=lat1
        latency_list.append(latency)
        nceEst_hci_master_val = nceEst_hci_master(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
        data.at[index,'nce'] = nceEst_hci_master_val
        nceEst_hci_slave_val = nceEst_hci_slave(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
        data.at[index,'nce_spork'] = nceEst_hci_slave_val
        nceEst_RTT_val = nceEst_RTT(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
        data.at[index,'nce_RTT'] = nceEst_RTT_val
        nceEst_kalai_val = nceEst_kalai(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'],controlParams['nperi'])
        data.at[index,'nce_kalai'] = nceEst_kalai_val
        nceEst_hci_master_list.append(nceEst_hci_master_val)
        nceEst_hci_slave_list.append(nceEst_hci_slave_val)
        nceEst_RTT_list.append(nceEst_RTT_val)
        nceEst_kalai_list.append(nceEst_kalai_val)
        window = window+1
        #print(nceEst_kalai_list)
        max_lat = max(latency_list)
        min_lat = min(latency_list)
        min_max_diff = max_lat-min_lat
        data.at[index,'latency'] = latency
        data.at[index,'lat_min_max_diff'] = min_max_diff
        latTh_latMax_diff = max_lat - controlParams['expectedLat']
        data.at[index,'latTh_latMax_diff'] = latTh_latMax_diff
        latTh_diff = latency - controlParams['expectedLat']
        #print('Latency fo')
        data.at[index,'latTh_lat_diff'] = latTh_diff
        data.at[index,'est_ci'] = controlParams['ci']
        data.at[index,'pow'] = power
        ciFinal=controlParams['ci']
        #--------------- spork's algorithm----------------------
        if(window == windowSize):
            window=0
            badLat = [e for e in latency_list if e>controlParams['expectedLat']]
            latCount = len(badLat)
            print('bad latCount :',badLat)
            lqf=len(badLat)/len(latency_list)
            power=readPower()
            data.at[index,'pow'] = power
            '''pow_avg = np.average(pow_list)
            data.at[index,'pow_avg'] = pow_avg'''
            nceEst_hci_master_max = max(nceEst_hci_master_list)
            data.at[index,'nce_all'] = nceEst_hci_master_max
            nceEst_hci_slave_max = max(nceEst_hci_slave_list)
            data.at[index,'nce_spork_all'] = nceEst_hci_slave_max
            nceEst_RTT_max = max(nceEst_RTT_list)
            data.at[index,'nce_RTT_all'] = nceEst_RTT_max
            nceEst_kalai_max = max(nceEst_kalai_list)
            data.at[index,'nce_kalai_all'] = nceEst_kalai_max
            #print(nceEst_hci_slave_max)
            print(nceEst_hci_master_list)
            print(nceEst_hci_slave_list)
            print(nceEst_RTT_list)
            print(nceEst_kalai_list)
            ciEst_hci_master_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_master_max)
            data.at[index,'nce_ci'] = ciEst_hci_master_val
            ciEst_hci_slave_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_slave_max)
            data.at[index,'ci_spork'] = ciEst_hci_slave_val
            ciEst_RTT_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_RTT_max)
            data.at[index,'ci_RTT'] = ciEst_RTT_val
            ciEst_kalai_val = estimateCI_kalai(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_master_max,controlParams['nperi'])
            data.at[index,'ci_kalai'] = ciEst_kalai_val
            if(lqf > 0.1):
                # increase link quality by allowing more retransmissions at the cost of increased power
                currentCI = controlParams['ci']
                if (not (currentCI in blacklistCI) and lqf>0.1): #0.05 during interference
                    blacklistCI.append(currentCI)
                    print ("CI added to blacklist:",currentCI)
                if(optSize<(len(ci)-1)):
                    optSize = optSize+1
                estCI=ci[optSize]
                if(estCI in blacklistCI and optSize<(len(ci)-1)):
                    optSize = optSize+1
                    estCI=ci[optSize]
                if(estCI in blacklistCI and optSize<(len(ci)-1)):
                    optSize = optSize+1
                    estCI=ci[optSize]
                print('estimatedCI:',estCI)
            if(lqf < 0.1): #use 0 with interference 0.1 without interference
                # Decrease link quality by allowing less retransmissions at the cost of decreased power
                if(optSize>0):
                    optSize = optSize-1
                estCI=ci[optSize]
                if(estCI in blacklistCI and optSize>0):
                    optSize = optSize-1
                    estCI=ci[optSize]
                if(estCI in blacklistCI and optSize>0):
                    optSize = optSize-1
                    estCI=ci[optSize]
                print('estimatedCI:',estCI)
            #print(latency)
            print(ciEst_hci_master_val)
            print(ciEst_hci_slave_val)
            print(ciEst_RTT_val)
            print(ciEst_kalai_val)
            if(ciFinal<controlParams['expectedLat']/2 and not (estCI in blacklistCI)):
                ciFinal=estCI
            if(ciFinal<=8):
                ciFinal = 8
            #if(opt and spork):
            if(controlParams['ci']!=ciFinal):
                print('estimated connection intervals are ciEst_hci_master_val:',ciEst_hci_master_val,',ciEst_hci_slave_val:',ciEst_hci_slave_val,',ciEst_RTT_val:',ciEst_RTT_val,',ciEst_kalai_val:',ciEst_kalai_val)
                data = updateCI(ciFinal,data)
            nceEst_hci_master_list = []
            nceEst_hci_slave_list = []
            nceEst_RTT_list = []
            nceEst_kalai_list = []
            latency_list = []
        data.at[index,'finished'] = True
        if(index%1==0):
            data.to_csv('template_ci_algo_21stMay_5Devices_Data_Estimation_120_1.csv', encoding='utf-8', index=False)
            index=index+1
    #print('reading from uart')
    time.sleep(1)

   ind  finished  dpl  att_mtu  tp  phy  ci_initial  est_ci  pow  pow_avg  \
0    1       NaN  251      247   0    2         7.5       8  NaN      NaN   
1    2       NaN  251      247   0    2         7.5       9  NaN      NaN   
2    3       NaN  251      247   0    2         7.5      10  NaN      NaN   
3    4       NaN  251      247   0    2         7.5      11  NaN      NaN   
4    5       NaN  251      247   0    2         7.5      12  NaN      NaN   

   ...  nce_ci  nce_spork  nce_spork_all  ci_spork  nce_RTT  nce_RTT_all  \
0  ...     NaN        NaN            NaN       NaN      NaN          NaN   
1  ...     NaN        NaN            NaN       NaN      NaN          NaN   
2  ...     NaN        NaN            NaN       NaN      NaN          NaN   
3  ...     NaN        NaN            NaN       NaN      NaN          NaN   
4  ...     NaN        NaN            NaN       NaN      NaN          NaN   

   ci_RTT  nce_kalai  nce_kalai_all  ci_kalai  
0     NaN        NaN            

bad latCount : [123, 123, 128, 120, 113]
[154.57881239]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1,

bad latCount : [108, 103, 128, 103, 128, 125, 108, 113]
[177.58496967]
[2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
#spork algo
index=0
data = pd.read_csv('template_ci_algo_13thMay_5Devices_Spork_Estimation_esp32_wi.csv')
'''for index, row in data.iterrows():
    if(data.at[index,'finished']==False):
        break'''
print(data.head(5))
data=data.astype(object)
optdata = pd.read_csv('model_input_5.csv')
nceEst_hci_master_list = []
nceEst_hci_slave_list = []
nceEst_RTT_list = []
nceEst_kalai_list = []
latency_list = []
pow_list = []
ci=[]
power=readPower()
blacklistCI=[]
expRet = 2
opt=False
optSize = 0
ind=0
estCI=8
#ppk2_test.start_measuring()
nrf.flushInput()
spork=False
first=True
time.sleep(5)
df = CIOptimizer_data(controlParams['dpl'],controlParams['nperi'],controlParams['expectedLat'],0,optdata)
print(df)
df = df.sort_values(['nce'])
ci=df['ci'].unique()
print(ci)
print('*****************')
while index<=1000:
    time.sleep(1)
    lat = np.array([])
    lat = nrf.readline()
    if (lat != b'' or lat!=''):
        test = (lat.decode()).strip("x00\n")
        x = test.split(",")
        print(float(x[4].strip("\x00")))
        #print(bytes([12]))
        #lat1 = np.array([test])
        #lat1 = lat1.astype(np.int_)
        lat1 = int(x[0].strip("\x00"))
        #print(lat1)
        latency=lat1
        #print(latency)
        latency_list.append(latency)
        nceEst_hci_master_val = nceEst_hci_master(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
        data.at[index,'nce'] = nceEst_hci_master_val
        nceEst_hci_slave_val = nceEst_hci_slave(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
        data.at[index,'nce_spork'] = nceEst_hci_slave_val
        nceEst_RTT_val = nceEst_RTT(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'])
        data.at[index,'nce_RTT'] = nceEst_RTT_val
        nceEst_kalai_val = nceEst_kalai(latency,controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],controlParams['ci'],controlParams['nperi'])
        data.at[index,'nce_kalai'] = nceEst_kalai_val
        nceEst_hci_master_list.append(nceEst_hci_master_val)
        nceEst_hci_slave_list.append(nceEst_hci_slave_val)
        nceEst_RTT_list.append(nceEst_RTT_val)
        nceEst_kalai_list.append(nceEst_kalai_val)
        window = window+1
        #print(nceEst_kalai_list)
        max_lat = max(latency_list)
        min_lat = min(latency_list)
        min_max_diff = max_lat-min_lat
        data.at[index,'latency'] = latency
        data.at[index,'lat_min_max_diff'] = min_max_diff
        latTh_latMax_diff = max_lat - controlParams['expectedLat']
        data.at[index,'latTh_latMax_diff'] = latTh_latMax_diff
        latTh_diff = latency - controlParams['expectedLat']
        #print('Latency fo')
        data.at[index,'latTh_lat_diff'] = latTh_diff
        data.at[index,'est_ci'] = controlParams['ci']
        data.at[index,'pow'] = power
        ciFinal=controlParams['ci']
        #--------------- spork's algorithm----------------------
        if(window == windowSize):
            window=0
            badLat = [e for e in latency_list if e>controlParams['expectedLat']]
            latCount = len(badLat)
            print('bad latCount :',badLat)
            lqf=len(badLat)/len(latency_list)
            power=readPower()
            data.at[index,'pow'] = power
            '''pow_avg = np.average(pow_list)
            data.at[index,'pow_avg'] = pow_avg'''
            nceEst_hci_master_max = max(nceEst_hci_master_list)
            data.at[index,'nce_all'] = nceEst_hci_master_max
            nceEst_hci_slave_max = max(nceEst_hci_slave_list)
            data.at[index,'nce_spork_all'] = nceEst_hci_slave_max
            nceEst_RTT_max = max(nceEst_RTT_list)
            data.at[index,'nce_RTT_all'] = nceEst_RTT_max
            nceEst_kalai_max = max(nceEst_kalai_list)
            data.at[index,'nce_kalai_all'] = nceEst_kalai_max
            #print(nceEst_hci_slave_max)
            print(nceEst_hci_master_list)
            print(nceEst_hci_slave_list)
            print(nceEst_RTT_list)
            print(nceEst_kalai_list)
            ciEst_hci_master_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_master_max)
            data.at[index,'nce_ci'] = ciEst_hci_master_val
            ciEst_hci_slave_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_slave_max)
            data.at[index,'ci_spork'] = ciEst_hci_slave_val
            ciEst_RTT_val = estimateCI_spork(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_RTT_max)
            data.at[index,'ci_RTT'] = ciEst_RTT_val
            ciEst_kalai_val = estimateCI_kalai(controlParams['expectedLat'],controlParams['max_ce'],controlParams['payload'],controlParams['att_mtu'],nceEst_hci_master_max,controlParams['nperi'])
            data.at[index,'ci_kalai'] = ciEst_kalai_val
            if(lqf > 0.1):
                # increase link quality by allowing more retransmissions at the cost of increased power
                currentCI = controlParams['ci']
                if (not (currentCI in blacklistCI) and lqf>0.15):
                    blacklistCI.append(currentCI)
                    print ("CI added to blacklist:",currentCI)
                if(optSize<(len(ci)-1)):
                    optSize = optSize+1
                estCI=ci[optSize]
                if(estCI in blacklistCI and optSize<(len(ci)-1)):
                    optSize = optSize+1
                    estCI=ci[optSize]
                if(estCI in blacklistCI and optSize<(len(ci)-1)):
                    optSize = optSize+1
                    estCI=ci[optSize]
                print('estimatedCI:',estCI)
            if(lqf == 0):
                # Decrease link quality by allowing less retransmissions at the cost of decreased power
                if(optSize>0):
                    optSize = optSize-1
                estCI=ci[optSize]
                if(estCI in blacklistCI and optSize>0):
                    optSize = optSize-1
                    estCI=ci[optSize]
                if(estCI in blacklistCI and optSize>0):
                    optSize = optSize-1
                    estCI=ci[optSize]
                print('estimatedCI:',estCI)
            #print(latency)
            print(ciEst_hci_master_val)
            print(ciEst_hci_slave_val)
            print(ciEst_RTT_val)
            print(ciEst_kalai_val)
            if(ciFinal<controlParams['expectedLat']/2 and not (estCI in blacklistCI)):
                ciFinal=estCI
            if(ciFinal<=8):
                ciFinal = 8
            #if(opt and spork):
            if(controlParams['ci']!=ciFinal):
                print('estimated connection intervals are ciEst_hci_master_val:',ciEst_hci_master_val,',ciEst_hci_slave_val:',ciEst_hci_slave_val,',ciEst_RTT_val:',ciEst_RTT_val,',ciEst_kalai_val:',ciEst_kalai_val)
                data = updateCI(ciFinal,data)
            nceEst_hci_master_list = []
            nceEst_hci_slave_list = []
            nceEst_RTT_list = []
            nceEst_kalai_list = []
            latency_list = []
        data.at[index,'finished'] = True
        if(index%1==0):
            data.to_csv('template_ci_algo_13thMay_5Devices_Data_Estimation_esp32_wi.csv', encoding='utf-8', index=False)
            index=index+1
    #print('reading from uart')
    time.sleep(1)